# Question 2: Evaluate the importance of all these features, find the most important one and analyze why is it so important.
## Methodology: 
### Random Forest

In [1]:
# Import lib
# ===========================================================
from sklearn import tree
# from imblearn.over_sampling import SMOTE
# from imblearn.over_sampling import SMOTENC
import pandas as pd
import time
import sys
import csv
from datascience import *
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
plt.style.use('fivethirtyeight')
import collections
import math
from tqdm import tqdm
from time import sleep
# from RandomForestFunctions import *

### 1. Initialization

In [6]:
# Initialize useful data
# ===========================================================
df = pd.read_csv('clinvar_conflicting_clean.csv', low_memory=False)
df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']] = df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']].fillna(value=0)
df_zero = df.loc[df['CLASS'] == 0]
df_zero = df_zero.sample(n=10000)
df_one = df.loc[df['CLASS'] == 1]
df_one = df_one.sample(n=10000)

df = pd.concat([df_zero, df_one])
df = df.sample(n = df.shape[0])
all_rows = df.values.tolist()
row_num = len(all_rows)

### 2. Split Train Test

In [7]:
# Divide whole dataset into training set and testing set
# ===========================================================
training_percentage = 0.01  # percent of partition of training dataset
training_size = int(row_num * training_percentage)
testing_size = row_num - training_size
training_attribute = list(df.columns)[: -1]# should exclude 'CLASS'
training_data = all_rows[: training_size]  # training data should include header row
testing_data = all_rows[training_size: ]   # testing data don't need to include header row

### 3. Variable Init

In [39]:
# number of attributes to use
# ===========================================================
rand_attribute_subset_len = 10  # can't too small, or it'll be hard for the tree to converge
final_acc_list = []
# final_time_list = []
final_attribute = []
forest_size = 2
test_times = 4

### 4. Random Forest Structure

In [40]:
# Decision stump part for Random Forest
# ===========================================================
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

# === LeafNode is the prediction result of this branch ===
class LeafNode:
    def __init__(self, rows):
        labels = [row[-1] for row in rows]
        self.prediction = collections.Counter(labels)

# === DecisionNode is an attribute / question used to partition the data ===
class DecisionNode:
    def __init__(self, question = None, left_branch = None, right_branch = None):
        self.question = question
        self.left_branch = left_branch
        self.right_branch = right_branch
    
class DecisionTree:
    def __init__(self, all_attribs, training_attribs, training_data, method = "CART"):
        self.training_attribs = training_attribs     # takein attribute and data separately
        self.all_attribs = all_attribs
        self.train = training_data
        self.row_num = len(self.train)
        self.attribute_colNums = [all_attribs.index(att) for att in training_attribs]
        self.method = method.upper()            # convert to upper case for general use
        self.labels = self.uniq_val(-1)
        if self.method not in ["C4.5", "CART", "HYBRID"]:
            print("Error: Please choose a valid method!")
            return None
        self.root = self.build_tree(self.train)
    
    def uniq_val(self, column):
        return set([self.train[i][column] for i in range(len(self.train))])
    
    # === when raising a question ===
    # if it's a categorical attribute, we simply iterate all categories
    # if it's a numeric attribute, we iterate the set of possible numeric values 
    class Question:
        def __init__(self, column, ref_value, attribute):
            self.column = column
            self.ref_value = ref_value if ref_value else "None"
            self.attri = attribute

        def match(self, row):
            if is_numeric(self.ref_value):
                try:
                    return row[self.column] >= self.ref_value
                except:
                    print("Error occured in ", row)
                    return True
            else:
                return row[self.column] == self.ref_value

        def __repr__(self):
            operand = ">=" if is_numeric(self.ref_value) else "=="
            return "Is %s %s %s?" % (self.attri[self.column], operand, str(self.ref_value))
    
    # === Method 1 - C4.5 ===
    def entropy(self, rows):
        # === Bits used to store the information ===
        labels = [row[-1] for row in rows]
        frequency = collections.Counter(labels).values()
        pop = sum(frequency)
        H = 0
        for f in frequency:
            p = f / pop
            H -= p * math.log(p, 2)
        return H
    
    # === Method 2 - CART ===
    def gini(self, rows):
        # === Probability of misclassifying any of your label, which is impurity ===
        labels = [row[-1] for row in rows]
        frequency = collections.Counter(labels).values()
        pop = sum(frequency)
        gini = 1
        for f in frequency:
            p = f / pop
            gini -= p ** 2
        return gini
    
    # === Calculate Gain Info ===
    # I'm actually returning the gain info reduction
    def info(self, branches, root):
        # === Objective: to find the best question which can maximize info ===
        root_size = float(len(root))
        if self.method == "C4.5":  # Here I pick the GainRatio Approach
            root_uncertainty = self.entropy(root)
            gain_info = root_uncertainty
            split_info = 0
            for branch in branches:
                if not branch: continue
                gain_info -= len(branch) / root_size * self.entropy(branch)
                split_info -= float(len(branch)) / root_size * math.log(float(len(branch)) / root_size)
#                 print(gain_info, split_info)
            return gain_info / split_info
        elif self.method == "CART":
            root_uncertainty = self.gini(root)
            gain_info = root_uncertainty
            for branch in branches:
                if not branch: continue
                gain_info -= len(branch) / root_size * self.gini(branch)
            return gain_info
        elif self.method == "HYBRID":
            pass
        pass
    
    # Divide rows according to the question to true_rows and false_rows
    # === Here I only do Binary Partitions ===
    def partition(self, rows, question):
        true_rows = []
        false_rows = []
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows
    
    def find_best_question(self, rows):
        max_info_attenuation = 0
        best_question = self.Question(self.attribute_colNums[0], self.train[0][self.attribute_colNums[0]], self.all_attribs)
        # === Iterate through all question candidates ===
        # === TODO: Maybe Iteration here can be optimized ===
        for col in self.attribute_colNums: # minus 1 to avoid using the label as attribute
            ref_candidates = self.uniq_val(col)
            for ref_value in ref_candidates:
                if ref_value == "null" or not isinstance(ref_value, str) and np.isnan(ref_value): continue # avoid using null values to generate a question
                q = self.Question(col, ref_value, self.all_attribs)
                temp_true_rows, temp_false_rows = self.partition(rows, q)
                temp_info_attenuation = self.info([temp_true_rows, temp_false_rows], rows)
                if temp_info_attenuation >= max_info_attenuation:
                    max_info_attenuation = temp_info_attenuation
                    best_question = q
        return max_info_attenuation, best_question
    
    # === Input rows of data with attributes and labels ===
    def build_tree(self, rows):
        # === Assign all rows as root of the whole decision tree ===
        # === We have met the leaf node if gini(rows) is 0 or no question candidates left ===
        gain_reduction, q = self.find_best_question(rows)
        # gain here is actually info reduction
#         if gain_reduction <= 0.001:
        if self.gini(rows) <= 0.48:
            return LeafNode(rows)
        true_rows, false_rows = self.partition(rows, q)
        # === Recursion after we have found a optimal question ===
        return DecisionNode(q, self.build_tree(true_rows), self.build_tree(false_rows))
    
    # === Input a row of data with attributes (and no label), predict its label with our decision tree ===
    # === Actually it can contain a label, we just don't use it ===
    # === walk down the decision tree until we reach the leaf node ===
    def classify(self, row, node):
        if isinstance(node, LeafNode):
#             print("===", node.prediction)
            return node.prediction
        
        if node.question.match(row):
#             print(node.question, True)
            return self.classify(row, node.left_branch)
        else:
#             print(node.question, False)
            return self.classify(row, node.right_branch)
    
    def print_tree(self, node, spacing=""):
        # Base case: we've reached a leaf
        if isinstance(node, LeafNode):
            print (spacing + "Predict", node.prediction)
            return

        # Print the question at this node
        print (spacing + str(node.question))

        # Call this function recursively on the true branch
        print (spacing + '--> True:')
        self.print_tree(node.left_branch, spacing + "  ")

        # Call this function recursively on the false branch
        print (spacing + '--> False:')
        self.print_tree(node.right_branch, spacing + "  ")
    
'''    def test(self):
        for i in range(self.column_num):
            q = self.Question(i, self.train[1][i], self.attribute)
            print(q)
            print(q.match(1))'''
    
def bootstrapped_dataset(rows, size):
    n = len(rows)
    bootstrapped_rows = []
    # here i should pick rand_idx with replacement, which is bootstrapping
    rand_idx = np.random.choice(n, size, replace=True)
    for i in rand_idx:
        bootstrapped_rows.append(rows[i])
    return bootstrapped_rows

### 5. Train Random Forest for each Fixed Attribute except 'CLASS'

In [41]:
for fixed_attribute in training_attribute:
	remaining_attribute = list(training_attribute)
	remaining_attribute.remove(fixed_attribute)

	print("Training for: %s" % fixed_attribute)

	start = time.time()

	final_acc = 0

	for pp in range(test_times):

		# Training Random Forest
		# ===========================================================
		random_forest = []
		
		for i in range(forest_size):
			rand_attribute_subset = np.random.choice(a=remaining_attribute, size=rand_attribute_subset_len - 1)
			rand_attribute_subset = np.append(rand_attribute_subset, fixed_attribute)
			# print(rand_attribute_subset)
			training_data = bootstrapped_dataset(all_rows, training_size)
			tree = DecisionTree(training_attribute, rand_attribute_subset, training_data, "CART")
# 			tree.print_tree(tree.root)
			random_forest.append(tree)

		end = time.time()
		print("Random Forest Trained! Time: %.03fs" % ((end - start) / 10))

		# Testing Random Forest, Computing TN, TP, FN, FP, etc.
		# ===========================================================

		CMap = {0: 'TN', 1: 'FN', 2: 'FP', 3: 'TP'}
		cutoff = 0.5
		Confusion = {'TN': 0, 'FN': 0, 'FP': 0, 'TP': 0}
		for row in testing_data:
			true_rate_forest = 0
			for tree_i in random_forest:

				# prediction is a counter of label 1 and 0
				pred_counter = tree_i.classify(row, tree_i.root)
				true_rate_tree = pred_counter.get(1, 0) / (pred_counter.get(1, 0) + pred_counter.get(0, 0) + 0.00000001)
				true_rate_forest += true_rate_tree
			true_rate_forest /= forest_size
			true_pred = 1 if true_rate_forest >= cutoff else 0
			indicator = (true_pred << 1) + row[-1]

			# accordingly update confusion matrix
			Confusion[CMap[indicator]] += 1
			
		final_acc += (Confusion['TP'] + Confusion['TN']) / sum(Confusion.values())

	end = time.time()
	print("Random Forest Tested! Time: %.03fs" % ((end - start) / test_times))
	final_acc /= test_times
	print(final_acc, ((end - start) / test_times))
	final_acc_list.append(final_acc)
	final_attribute.append(fixed_attribute)

Training for: CHROM
Random Forest Trained! Time: 0.155s
Random Forest Trained! Time: 0.258s
Random Forest Trained! Time: 0.411s
Random Forest Trained! Time: 0.511s
Random Forest Tested! Time: 1.298s
0.5820075757575758 1.2975522875785828
Training for: POS
Random Forest Trained! Time: 0.039s
Random Forest Trained! Time: 0.178s
Random Forest Trained! Time: 0.354s
Random Forest Trained! Time: 0.452s
Random Forest Tested! Time: 1.156s
0.5853787878787878 1.1560677886009216
Training for: REF
Random Forest Trained! Time: 0.051s
Random Forest Trained! Time: 0.273s
Random Forest Trained! Time: 0.384s
Random Forest Trained! Time: 0.551s
Random Forest Tested! Time: 1.409s
0.5154166666666666 1.4090960025787354
Training for: ALT
Random Forest Trained! Time: 0.141s
Random Forest Trained! Time: 0.275s
Random Forest Trained! Time: 0.336s
Random Forest Trained! Time: 0.396s
Random Forest Tested! Time: 1.012s
0.587260101010101 1.0123035311698914
Training for: AF_ESP
Random Forest Trained! Time: 0.101s
Ra

In [42]:
weight = Table()
weight = weight.with_columns('Attribute', final_attribute, 'Acc', final_acc_list)
weight.sort(1, descending=True)

Attribute,Acc
AF_EXAC,0.641288
CADD_PHRED,0.632146
AF_ESP,0.63096
CLNDISDB,0.620606
AF_TGP,0.614217
INTRON,0.612929
Codons,0.610189
CDS_position,0.606843
Allele,0.599811
CLNHGVS,0.597197


# Below are script cells

In [14]:
tree.print_tree(tree.root)

Is SIFT == tolerated?
--> True:
  Is POS >= 32913320?
  --> True:
    Predict Counter({1: 15, 0: 5})
  --> False:
    Predict Counter({0: 6, 1: 1})
--> False:
  Is POS >= 47412703?
  --> True:
    Predict Counter({0: 65, 1: 38})
  --> False:
    Is POS >= 43503710?
    --> True:
      Predict Counter({1: 5})
    --> False:
      Is LoFtool >= 0.13699999999999998?
      --> True:
        Predict Counter({1: 15, 0: 10})
      --> False:
        Predict Counter({0: 26, 1: 14})
